# 1) Daten laden

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Wetter- und Luftqualitätsdaten") \
    .getOrCreate()

## 1.1) Woher kommen die Daten und in welcher Form liegen sie vor?

#### Wetterdaten Zürich und St. Gallen
Die Wetterdaten von Zürich und St. Gallen stammen vom Bundesamt für Meteorologie und Kimatologie - MeteoSchweiz. Abrufbar unter folgendem Link: xxx

Es handelt sich hierbei um tägliche Messwerte, die als CSV-Dateien im UFT-8 Format vorliegen. Das Datumsfeld trägt den Namen *reference_timestamp* und ist im Format DD.MM.YYYY HH:MM als Zeichenkette gespeichert. Die Dateien umfassen Messwerte aus zwei Wetterstationen: 

* Zürich Fluntern (SMA)
* St. Gallen (STG)

#### Luftqualitätsdaten Zürich und St. Gallen

Die Luftqualitätsdaten stammen von der Plattform OSTLUFT abrufbar unter folgendem Link: https://www.ostluft.ch/messwerte/datenabfrage, welche von versch. kantonalen Fachstellen getragen wird, darunter das Amt für Abfall, Wasser, Energie und Luft (AWEL) des Kantons Zürich sowie das Amt für Umwelt des Kantons St. Gallen. 

Die Daten wurden über die offizielle Datenabfrage-Schnittstelle heruntergeladen und liegen als CSV-Dateien im UFT-8 Format vor. Es handelt sich um aggregierte Tagesmittelwerte wichtiger Parameter für Luftschadstoffe.

Das Datumsfeld heisst *Startzeit* udn ist im Format TT.MM.YYYY HH:MM vor. Die Messwerte stammen von zwei festen Luftmessstationen:

* **Zürich**: Station Zch_Stampfenbachstr
* **St. Gallen**: Station StG_St.Leonhard-Str

In [3]:
## Schritt 1: Wetterdaten einlesen

# Zürich
df_zurich = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .option("sep", ";") \
    .csv("data/weather/zurich/ogd-smn_sma_d_historical.csv")

# St. Gallen
df_stgallen = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .option("sep", ";") \
    .csv("data/weather/stgallen/ogd-smn_stg_d_historical.csv")

In [4]:
from pyspark.sql.functions import lit

# Zürich
df_air_zurich = spark.read \
    .option("header", True) \
    .option("sep", ";") \
    .option("inferSchema", True) \
    .csv("data/air_quality/airquality_zurich.csv") \
    .withColumn("location", lit("Zürich"))

# St. Gallen
df_air_stgallen = spark.read \
    .option("header", True) \
    .option("sep", ";") \
    .option("inferSchema", True) \
    .csv("data/air_quality/airquality_stgallen.csv") \
    .withColumn("location", lit("St. Gallen"))

# Zusammenführen
df_air = df_air_zurich.unionByName(df_air_stgallen)

---------------------------------------------------------

## 1.2) Pivotierung der Luftqualitätsdaten

Das ursprüngliche Format der Luftqualitätsdaten lag im "Long Format" vor. In dieser Struktur entsprach jede Zeile einen einzelnen Messswert, der durch Datum, Standort sowie Schadstoffparameter bestimmt war. Um die Daten jedoch effizient analysieren und mit den Wetterdaten veknüpfen zu könenn, wird eine Pivotierung vorgenommen. Das Ergebnis ist ein "Wide Format", bei dem jeder schadstoff als eigene Spalte dargestellt ist. So enthält jede Zeile alle relevanten Messwerte eines Tages für einen bestimmten Standort.

In [5]:
from pyspark.sql.functions import to_date, col, first, when

# Bereinigung
df_air_clean = df_air

# Pivotierung
from pyspark.sql.functions import to_date, col

df_air_clean = df_air_clean.withColumn(
    "date", to_date(col("Startzeit"), "dd.MM.yyyy HH:mm")
)

In [7]:
from pyspark.sql.functions import col, first

df_luft_pivot = df_air_clean.groupBy("date", "location").agg(
    first(col("PM10")).alias("PM10"),
    first(col("`PM2.5`")).alias("PM2.5"),
    first(col("Ozon")).alias("Ozon"),
    first(col("CO")).alias("CO"),
    first(col("NO2")).alias("NO2"),
    first(col("NO")).alias("NO")
)

---------------------------------------------------------

## 1.3) Datumsformat vereinheitlichen

Um die Verarbeitung sowie Verknüpfung der Wetter- und Luftqualitätsdaten zu ermöglichen, wird das Datumsformat in beiden Datsätzen vereinheitlicht. Die ursprüngliche Datums- und Zeitangabe liegen in unterschiedlcihen Formate vor und zwar:

* **Wetterdaten**: DD.MM.YYYY HH:mm (reference_timestamp)
* **Luftqualitätsdaten**: DD.MM.YYYY (Startzeit)

Durch die Konvertierung in ein einheitliches Datums- und Zeitangabeformat wird sichergestellt, dass beide Quellen korrekt über das Datum gejoint werden können.

In [9]:
from pyspark.sql.functions import to_date, col

# Wetterdaten: Datum umwandeln
df_zurich = df_zurich.withColumn(
    "date", to_date(col("reference_timestamp"), "dd.MM.yyyy HH:mm")
)
df_stgallen = df_stgallen.withColumn(
    "date", to_date(col("reference_timestamp"), "dd.MM.yyyy HH:mm")
)

---------------------------------------------------------

## 1.4) Standortbezeichnung vereinheitlichen

Damit Wetter- und Luftqualitätsdaten korrekt zusammengeführt werden können, ist es notwendig, dass die Bezeichnung der Standorte gleich sind. Aktuell haben die Rohdaten noch unterschiedliche Standortbezeichnungen:

* **Standort in Wetterdaten**: In der Spalte *station_abbr* finden sich SMA für Zürich und STG für St. Gallen
* **Standort in Luftqualitätsdaten**: In der Spalte *location* finden wir Zch_Stampfenbachstr für Zürich und StG_St.Leonhard-Str für St. Gallen

Damit der Join erfolgreich passieren wird, wird eine neue Spalte *location* erstellt, in der die Standorte konsistent mit **"Zürich"** und **"St. Gallen"** benannt werden können.

In [10]:
from pyspark.sql.functions import when, col

# Standort Wetterdaten standardisieren
if "station_abbr" in df_zurich.columns:
    df_weather_std = df_zurich.withColumn(
        "location",
        when(col("station_abbr") == "SMA", "Zürich")
        .when(col("station_abbr") == "STG", "St. Gallen")
        .otherwise("Andere")
    )
else:
    print("Spalte 'station_abbr' nicht gefunden in df_zurich!")
    df_weather_std = df_zurich

# Standort Luftqualitätsdaten standardisieren
df_air_std = df_luft_pivot.withColumn(
    "location",
    when(col("location").startswith("Zch_"), "Zürich")
    .when(col("location").contains("St. Gallen"), "St. Gallen")
    .otherwise("Andere")
)

---------------------------------------------------------

## 1.5) Join Wetter- und Luftqualitätsdaten

Die Zusammenführung der Wetter- und Luftqualitätsdaten brauchen wir, damit wir später mit einem einheitlichen Datensatz weiter arbeiten können, der alle relevanten Merkmale für eine Modellierung beinhaltet. Die Wetter- und Luftqualitätsdaten sind grundsätzlich örtlich und zeitlich aufeinander abgestimtm worden und können nun miteinander verknüpft werden.

Ziel ist es mögliche Wettereinflüsse auf die Luftqualität zu analysieren und so ein prädiktives Modell zu trainieren. Durch den Join stehen uns pro Zeile sowohl meterologische als auch umweltrelevante Parameter pro Tag und Standort zur Verfügung.

In [14]:
from pyspark.sql.functions import year

df_air_std = df_air_std.filter((col("date").isNotNull()) & (year("date") > 1900))
df_weather_std = df_weather_std.filter((col("date").isNotNull()) & (year("date") > 1900))

In [15]:
df_joined = df_air_std.join(
    df_weather_std,
    on=["date", "location"],
    how="inner"
)

---------------------------------------------------------